In [31]:
import os, subprocess, json
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

class DenseEncoder(object):
        
        def xlm_roberta_base_encoder(self, input_text):
                # Load the tokenizer and model
                tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
                model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
                tokens = tokenizer.encode(input_text, add_special_tokens=True, max_length=512, truncation=True)
                input_ids = torch.tensor([tokens])
                with torch.no_grad():
                        outputs = model(input_ids)
                        dense_vector = outputs[0][0][0]  # Extract the dense vector from the model output

                # Convert the dense vector to a numpy array
                dense_vector_list = dense_vector.numpy().tolist()
                return dense_vector_list

        
        def document_encoder(self):
                input_file = os.path.join("/home/peng_luh/__git/search_l3s/search_l3s_search_srv", "datasets/mls-tasks/tasks.json")
                output_file = os.path.join("/home/peng_luh/__git/search_l3s/search_l3s_search_srv", "encodes/dense_encoder")
                
                with open(input_file) as f:
                        data = json.load(f)


                for d in data:
                        d["@id"] = d["id"]
                        d["id"] = int(d["id"].split("/")[-1])
                        d["vector"] = self.xlm_roberta_base_encoder(d["contents"])
                        d["contents"] = d["contents"] + "\n"

                        with open(f"{output_file}/task.jsonl", "w") as jsonl_file:
                                for d in data:
                                        json.dump(d, jsonl_file)
                                        jsonl_file.write('\n')
                return


In [22]:
# from .logic import DenseEncoder

dense_enc = DenseEncoder()

# dense_vec = dense_enc.xlm_roberta_base_encoder("Was is Elektrotechnik")

# print(dense_vec)


In [32]:
import os

dense_enc.document_encoder()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are ini

KeyboardInterrupt: 

In [ ]:
import subprocess

cmd2 = """
    python -m pyserini.index.faiss \
        --input /home/peng_luh/__git/search_l3s/search_l3s_search_srv/encodes/dense_encoder/xlm_roberta_base \
        --output /home/peng_luh/__git/search_l3s/search_l3s_search_srv/indexes/dense/hnsw \
        --hnsw
"""

subprocess.call(cmd2, shell=True)